In [ ]:
!pip install -r requirements.txt

In [171]:
!python3 train.py --k 5 --max_epochs 10 --batch_size 2

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packa

In [163]:
!python3 infer.py --desc '죄와 벌- 도스토옙스키,살해한다)를 살해하고 돈을 훔치며 방황하다 결국 사랑과 신 앞에 자신의 과오를 뉘우치고 새삶을 찾는다는 다분히 개과천선적인 이야기이다. 하지만 이 소설을 내내 편히 읽으면서 자연스럽게 파악하게 되는 것이 있는데 바로 라스콜리니코프와 대척점에...' 

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
### desc: 죄와 벌- 도스토옙스키,살해한다)를 살해하고 

### model arch

In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")
string = "왜 응급실만 들어가면 함흥차사인지, 빨리 진행이 잘 될 줄 알고 응급실을 갔는데 이건 오히려 왜 더 걸리는지 답답하셨던 분들을 위해 응급실의 입원까지의 과정에 대해 간략히 적어보았다. 하지만 이는 일반적인 상황이며 정말 초응급한 상황에서는 모든 것이 정신없이 지나간다."
model = AutoModelForMaskedLM.from_pretrained("klue/roberta-small", output_hidden_states =True)


In [2]:
lefts = [" ".join(["[MASK]"] * 5)]
rights = [string]
encodings = tokenizer(text=lefts,
                              text_pair=rights,
                              return_tensors="pt",
                              add_special_tokens=True,
                              truncation=True,
                              padding=True)

In [3]:
S_subword = model.forward(**encodings).logits
S_subword = S_subword[:, 1:5+1]
S_subword.size()

torch.Size([1, 5, 32000])

In [4]:
S_subword.size()

torch.Size([1, 5, 32000])

In [5]:
vocab = ['함흥차사', '마부작침', '독서망양', '군계일학', '대기만성', '수어지교', '개과천선', '조령모개', '다다익선', '백전백승']
mask_id = tokenizer.convert_tokens_to_ids("[MASK]")
pad_id = tokenizer.convert_tokens_to_ids("[PAD]")
encoded = tokenizer(text=vocab,
                    add_special_tokens=False,
                    padding='max_length',
                    max_length=5,  # set to k
                    return_tensors="pt")
input_ids = encoded['input_ids']
input_ids[input_ids == pad_id] = mask_id  # replace them with masks
input_ids

tensor([[29867,  2232,  2063,     4,     4],
        [29647,  2333,  2540,     4,     4],
        [ 6133,  2087,  2221,     4,     4],
        [  617,  2418,  2210,  2218,     4],
        [ 5889,  2154,  2047,     4,     4],
        [ 1295,  4379,  2120,     4,     4],
        [  558,  2145,  2337,  2020,     4],
        [ 1552,  2291,  2391,  2019,     4],
        [15871,  2427,  2020,     4,     4],
        [ 1132,  2165,  2353,  2164,     4]])

In [6]:
word2subs= input_ids
word2subs.size()

torch.Size([10, 5])

In [7]:
word2subs = word2subs.T.repeat(S_subword.shape[0], 1, 1)  # (|V|, K) -> (N, K, |V|)
word2subs.size()

torch.Size([1, 5, 10])

In [8]:
S_subword.gather(dim=-1, index=word2subs)

tensor([[[-3.1390, -4.3971,  1.1603,  0.8054,  2.6666,  3.3189,  2.9078,
           4.2063, -1.5453,  2.6153],
         [ 3.0248, -1.3976, -0.9941,  1.9440,  2.8471, -4.1056,  3.5755,
          -2.2962, -3.0727,  3.3772],
         [ 3.4585,  2.1410,  2.1075,  3.9764,  2.3541,  1.3856,  0.6869,
           2.5727,  2.9207,  0.9953],
         [-4.2341, -4.2341, -4.2341,  3.3528, -4.2341, -4.2341,  3.4979,
           3.1271, -4.2341,  0.4532],
         [-3.9272, -3.9272, -3.9272, -3.9272, -3.9272, -3.9272, -3.9272,
          -3.9272, -3.9272, -3.9272]]], grad_fn=<GatherBackward0>)

In [9]:
S_subword = S_subword.gather(dim=-1, index=word2subs).sum(dim=1)
S_subword

tensor([[ -4.8171, -11.8150,  -5.8877,   6.1515,  -0.2935,  -7.5625,   6.7409,
           3.6828,  -9.8587,   3.5137]], grad_fn=<SumBackward1>)

In [10]:
import torch
y = torch.tensor([2]).long()

In [11]:
from torch.nn import functional as F

F.cross_entropy(S_subword, y)

tensor(13.1246, grad_fn=<NllLossBackward0>)